<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Moscow</font></h1>



## Introduction

As a young professional living in Moscow I am currently renting an apartment, but I am considering buying my own apartment in the future. I like the neighborhood I currently live in and I want to find some similar neighborhoods for possible relocation.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Description of Data</a>

2. <a href="#item2">Methodology</a>
    
3. <a href="#item3">Segmenting by price</a>

4. <a href="#item4">Download and Explore Dataset</a>

5. <a href="#item5">Explore and Pre-Select Neighborhoods in Moscow</a>

6. <a href="#item6">Analyze Each Neighborhood</a>

7. <a href="#item7">Cluster Neighborhoods</a>

8. <a href="#item8">Examine Clusters</a>
    
9. <a href="#item9">Conclusion</a>
</font>
</div>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# import DBSCAN from clustering stage
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>

## Description of data

There are several sources of data used in this analysis.
1. Moscow geographical data on boroughs and neighborhoods
Gis-LAB, an independent resource on Moscow geo data, can be used to obtain a CSV file with neighborhoods, boroughs and geo data on the borders of the neighborhoods in latitudes and longitudes. That data will then have to be adjusted as for the clustering I will need a single point within the borders of the neighborhood for segmentation. 

2. Data on average price in different Moscow neighborhoods  
IRN, analytical center for real estate, has the data on average price per sq.m for neighborhoods in Moscow. That data will be used for pre-selecting neighborhoods based on price.

3. Foursquare API data on venues.
Having pre-selected neighborhoods data i will use a get request to get data on venues located near the neighborhood center. I will only need data on the venue type, as the neighborhoods will be clustered based on that.

## Methodology

In this notebook, I will use Foursquare API to explore neighborhoods in Moscow to find appropriate neighborhoods for relocation. I will pre-select the neighborhoods using DBSCAN algorithm based on price and then use the **explore** function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. I will use the *k*-means clustering algorithm to complete this task. Finally, i will use the Folium library to visualize the neighborhoods in Moscow and their emerging clusters and based on the clustering results choose the appropriate neighborhoods.

## Download and Explore Dataset

In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. 

Let's first download the file with names of boroughs and neighborhoods in Moscow and their geographical coordinates and have a look on it.

In [3]:
moscow_df=pd.read_csv('mo.csv')
moscow_df.head()

,WKT,NAME,OKATO,OKTMO,NAME_AO,OKATO_AO,ABBREV_AO,TYPE_MO
0,"MULTIPOLYGON (((36.8031012 55.4408329,36.80319...",Киевский,45298555,45945000,Троицкий,45298000,Троицкий,Поселение
1,"POLYGON ((37.4276499 55.7482092,37.4284863 55....",Филёвский Парк,45268595,45328000,Западный,45268000,ЗАО,Муниципальный округ
2,"POLYGON ((36.8035692 55.4516224,36.8045117 55....",Новофёдоровское,45298567,45954000,Троицкий,45298000,Троицкий,Поселение
3,"POLYGON ((36.9372397 55.2413907,36.9372604 55....",Роговское,45298575,45956000,Троицкий,45298000,Троицкий,Поселение
4,"POLYGON ((37.4395575 55.6273129,37.4401803 55....","""Мосрентген""",45297568,45953000,Новомосковский,45297000,Новомосковский,Поселение


To save space I can remove columns with unnecessary information.

In [4]:
moscow_df=moscow_df.drop(columns=['OKATO', 'OKTMO', 'NAME_AO', 'OKATO_AO','TYPE_MO'])

Proceeding with tidying up the data i will rename the columns for clarity and analyze the results by boroughs.

In [5]:
moscow_df.rename(columns={'NAME':'Neighborhood','ABBREV_AO':'Borough'},inplace=True)
moscow_df['Borough'].value_counts()

СВАО              17
ВАО               16
САО               16
ЮАО               16
ЗАО               13
ЮВАО              12
ЮЗАО              12
Новомосковский    11
Троицкий          10
ЦАО               10
СЗАО               8
ЗелАО              5
Name: Borough, dtype: int64

At this stage I understand, that I am mostly interested in certain boroughs, mostly located to the North or West of the City, closer to where I work. So let's remove the other boroughs from analysis.

In [6]:
boroughs=['СВАО','САО','ЗАО','ЦАО','СЗАО']
moscow_df = moscow_df[moscow_df.Borough.isin(boroughs)]

Checking the resulting shape of the dataframe.

In [7]:
moscow_df.shape

(64, 3)

Now we need to get a single central point for longitude and latitude for each neighborhood. 
Currently we have a list of all points representing the border of each neighborhood in the column WKT. I will get a mean of latitude and longitude for all of those points for my further analysis.

In [8]:
# define a function that takes mean of a string of coordinates divided by a comma
def coord(list):

    i=0
    row1=[]
    for coordinate in coordinates:
        r=coordinates[i].split(',')
        r=[float(r[0].replace('((',' ').replace('))',' ')),float(r[1].replace('((',' ').replace('))',' '))]
        row1=row1+r
        i=i+1
        latitude=sum((row1[::2]))/len(row1[::2])
        longitude=sum((row1[1::2]))/len(row1[1::2])
    return [latitude,longitude]

In [9]:
# create an empty table to store coordinates
df_lalg=pd.DataFrame(columns=['Latitude','Longitude'])
i=0
# run a cicle for all neighborhoods
while i<=moscow_df.shape[0]-1:
    coordinates=moscow_df['WKT'].iloc[i].split(' ')
    coordinates=coordinates[2:len(coordinates)-2]    
    lg=coord(coordinates)
    df_lalg=df_lalg.append({'Latitude':lg[0],'Longitude':lg[1]},ignore_index=True)
    i=i+1

In [10]:
# merge the two tables together
moscow_df.reset_index(drop=True,inplace=True)
moscow_df=moscow_df.merge(df_lalg,left_index=True,right_index=True)
# drop unnecessary columns 
moscow_df=moscow_df.drop(columns=['WKT'])

Let's check the resulting dataframe.

In [11]:
moscow_df.head()

,Neighborhood,Borough,Latitude,Longitude
0,Филёвский Парк,ЗАО,55.748708,37.474149
1,Внуково,ЗАО,55.612560,37.280081
2,Щукино,СЗАО,55.798118,37.475900
3,Дмитровский,САО,55.886669,37.529184
4,Ховрино,САО,55.865166,37.486793


In [12]:
moscow_df.shape

(64, 4)

## Segmenting by price

Let's start with downloading the file with the data on average price per sq.m. for neighborhoods in Moscow and add it to the original dataframe.

In [13]:
moscow_price=pd.read_excel('Price.xlsx')
moscow_price.head()

,Neighborhood,Price per sq.m,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,Остоженка,403710,NaN,NaN,NaN,NaN,NaN,NaN
1,Якиманка,384385,NaN,NaN,NaN,NaN,NaN,NaN
2,Арбат,370425,NaN,NaN,NaN,NaN,NaN,NaN
3,Киевский,361844,NaN,NaN,NaN,NaN,NaN,NaN
4,Тверской,345097,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
moscow_df=moscow_df.join(moscow_price[['Neighborhood','Price per sq.m']].set_index('Neighborhood'), on='Neighborhood')

In [15]:
moscow_df.shape

(64, 5)

Further I will check, whether all neighborhoods have been assigned a price per sq.m.

In [16]:
moscow_df['Price per sq.m'].isna().value_counts()

False    62
True      2
Name: Price per sq.m, dtype: int64

Since some of the neighborhoods are missing a price, I will assign an average price to those neighborhoods.

In [17]:
moscow_df.describe()

,Latitude,Longitude,Price per sq.m
count,64.000000,64.000000,62.000000
mean,55.802519,37.529175,196998.677419
std,0.073974,0.109717,58668.458329
min,55.612560,37.104304,137036.000000
25%,55.750567,37.473558,155005.750000
50%,55.806166,37.550749,183106.000000
75%,55.865264,37.610455,207106.000000
max,55.943791,37.701189,384385.000000


In [18]:
moscow_df.fillna(value=moscow_df['Price per sq.m'].mean(),inplace=True)

Let's check the results.

In [19]:
moscow_df['Price per sq.m'].isna().value_counts()

False    64
Name: Price per sq.m, dtype: int64

All set, the data is ready for clustering.

In [20]:
# transform the data for segmenting into a NumPy array 

price_clustering=moscow_df['Price per sq.m']
price_clustering=price_clustering.to_numpy()
price_clustering=np.reshape(price_clustering,(-1, 1))


In [21]:
# transform data for clustering
X= StandardScaler().fit_transform(price_clustering)

# compute DBSCAN
db = DBSCAN(eps=0.3, min_samples=10).fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))


Estimated number of clusters: 1
Estimated number of noise points: 13
Silhouette Coefficient: 0.713


In [22]:
# add clustering labels
moscow_df.insert(0, 'Cluster Labels', labels)

moscow_df.head() 

,Cluster Labels,Neighborhood,Borough,Latitude,Longitude,Price per sq.m
0,0,Филёвский Парк,ЗАО,55.748708,37.474149,207763.0
1,0,Внуково,ЗАО,55.612560,37.280081,137036.0
2,0,Щукино,СЗАО,55.798118,37.475900,188032.0
3,0,Дмитровский,САО,55.886669,37.529184,153984.0
4,0,Ховрино,САО,55.865166,37.486793,164663.0


Let's examine the results of clustering.

In [23]:
moscow_df['Cluster Labels'].value_counts()

 0    51
-1    13
Name: Cluster Labels, dtype: int64

In [24]:
moscow_df.head()

,Cluster Labels,Neighborhood,Borough,Latitude,Longitude,Price per sq.m
0,0,Филёвский Парк,ЗАО,55.748708,37.474149,207763.0
1,0,Внуково,ЗАО,55.612560,37.280081,137036.0
2,0,Щукино,СЗАО,55.798118,37.475900,188032.0
3,0,Дмитровский,САО,55.886669,37.529184,153984.0
4,0,Ховрино,САО,55.865166,37.486793,164663.0


DBSCAN algorithm identified 1 cluster and several noise points.
Now I will examine the noise points. 

In [25]:
moscow_df[moscow_df['Cluster Labels'] == -1]

,Cluster Labels,Neighborhood,Borough,Latitude,Longitude,Price per sq.m
22,-1,Раменки,ЗАО,55.696332,37.504112,244627.0
27,-1,Дорогомилово,ЗАО,55.738817,37.532958,256013.0
29,-1,Беговой,САО,55.781101,37.568215,242010.0
32,-1,Пресненский,ЦАО,55.762533,37.579689,309195.0
37,-1,Хамовники,ЦАО,55.740496,37.589018,330257.0
43,-1,Арбат,ЦАО,55.751186,37.595816,370425.0
45,-1,Тверской,ЦАО,55.769137,37.605589,345097.0
46,-1,Якиманка,ЦАО,55.729036,37.610110,384385.0
51,-1,Мещанский,ЦАО,55.776174,37.627336,276583.0
53,-1,Замоскворечье,ЦАО,55.730883,37.631416,312005.0


The results seem correct, as all of those neighborhoods are very expensive and should be excluded from further classification.
So I will drop those noise points and irrelevant columns.

In [26]:
moscow_df = moscow_df[moscow_df['Cluster Labels'] == 0].reset_index(drop=True)
moscow_df.drop(columns=['Cluster Labels','Price per sq.m'],inplace=True)

moscow_df.head()

,Neighborhood,Borough,Latitude,Longitude
0,Филёвский Парк,ЗАО,55.748708,37.474149
1,Внуково,ЗАО,55.612560,37.280081
2,Щукино,СЗАО,55.798118,37.475900
3,Дмитровский,САО,55.886669,37.529184
4,Ховрино,САО,55.865166,37.486793


In [27]:
print('The resulting dataframe has {} boroughs and {} neighborhoods.'.format(
        len(moscow_df['Borough'].unique()),
        moscow_df.shape[0]
    )
)

The resulting dataframe has 4 boroughs and 51 neighborhoods.


Let's view the selected neighborhoords on a map.

In [28]:
# create map of Moscow using latitude and longitude values
map_moscow = folium.Map(location=[55.751244, 37.618423], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(moscow_df['Latitude'], moscow_df['Longitude'], moscow_df['Borough'], moscow_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_moscow)  
    
map_moscow

#### Define Foursquare Credentials and Version

In [29]:
CLIENT_ID = 'SK0IS030B0SVY4PBGQXI5SE2DEMFCTPBUZLHLAE3N3F0NFNI' # your Foursquare ID
CLIENT_SECRET = 'TKJXRTX3FZFXSNGZEULGG0LVYWAPZQWDCTQM2T4KTZJM4W3L' # your Foursquare Secret
VERSION = '20201605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SK0IS030B0SVY4PBGQXI5SE2DEMFCTPBUZLHLAE3N3F0NFNI
CLIENT_SECRET:TKJXRTX3FZFXSNGZEULGG0LVYWAPZQWDCTQM2T4KTZJM4W3L


In [30]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

## Explore Neighborhoods in Moscow

Let's create a function to repeat the same process to all the neighborhoods in Moscow

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called *moscow_venues*.

In [32]:

moscow_venues = getNearbyVenues(names=moscow_df['Neighborhood'],
                                   latitudes=moscow_df['Latitude'],
                                   longitudes=moscow_df['Longitude']
                                  )



Филёвский Парк
Внуково
Щукино
Дмитровский
Ховрино
Можайский
Ново-Переделкино
Строгино
Молжаниновский
Митино
Куркино
Крылатское
Солнцево
Головинский
Южное Тушино
Северное Тушино
Покровское-Стрешнево
Хорошёво-Мнёвники
Очаково-Матвеевское
Тропарёво-Никулино
Левобережный
Фили-Давыдково
Войковский
Сокол
Западное Дегунино
Проспект Вернадского
Хорошёвский
Коптево
Аэропорт
Северный
Бескудниковский
Тимирязевский
Лианозово
Восточное Дегунино
Савёловский
Алтуфьевский
Марфино
Отрадное
Бутырский
Останкинский
Бибирево
Марьина Роща
Свиблово
Южное Медведково
Северное Медведково
Ростокино
Алексеевский
Бабушкинский
Ярославский
Лосиноостровский
Кунцево


Let's check the size of the resulting dataframe.

In [33]:
print(moscow_venues.shape)
moscow_venues.head()

(854, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Филёвский Парк,55.748708,37.474149,ПандаПарк Фили,55.750919,37.478342,Athletics & Sports
1,Филёвский Парк,55.748708,37.474149,Пляж,55.747616,37.474336,Beach
2,Филёвский Парк,55.748708,37.474149,Место для Барбекю,55.745042,37.471349,BBQ Joint
3,Филёвский Парк,55.748708,37.474149,Иван чайная,55.750337,37.479428,Tea Room
4,Филёвский Парк,55.748708,37.474149,Главная сцена,55.748931,37.480362,Concert Hall


Let's check how many venues were returned for each neighborhood.

In [34]:
moscow_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Алексеевский,28,28,28,28,28,28
Алтуфьевский,8,8,8,8,8,8
Аэропорт,9,9,9,9,9,9
Бабушкинский,34,34,34,34,34,34
Бескудниковский,7,7,7,7,7,7
Бибирево,6,6,6,6,6,6
Бутырский,80,80,80,80,80,80
Внуково,4,4,4,4,4,4
Войковский,48,48,48,48,48,48


Let's find out how many unique categories can be curated from all the returned venues.

In [35]:
print('There are {} uniques categories.'.format(len(moscow_venues['Venue Category'].unique())))

There are 205 uniques categories.


In [36]:
moscow_venues['Venue Category'].value_counts()

Supermarket                      37
Coffee Shop                      29
Pharmacy                         28
Gym / Fitness Center             26
Park                             25
Pizza Place                      24
Café                             22
Restaurant                       20
Bus Stop                         20
Clothing Store                   18
Cosmetics Shop                   17
Convenience Store                16
Fast Food Restaurant             16
Health Food Store                15
Sushi Restaurant                 13
Food & Drink Shop                12
Middle Eastern Restaurant        11
Bakery                           11
Caucasian Restaurant             10
Auto Workshop                    10
Sporting Goods Shop              10
Flower Shop                      10
Pet Store                        10
Gym                              10
Bookstore                         9
Salon / Barbershop                9
Hotel                             8
Photography Studio          

### Analyze each neighborhood

<a id='item3'></a>

In [37]:
# one hot encoding
moscow_onehot = pd.get_dummies(moscow_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
moscow_onehot['Neighborhood'] = moscow_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [moscow_onehot.columns[-1]] + list(moscow_onehot.columns[:-1])
moscow_onehot = moscow_onehot[fixed_columns]

moscow_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bakery,Bar,Bath House,Beach,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Blini House,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Burger Joint,Bus Line,Bus Station,Bus Stop,Butcher,Café,Camera Store,Campground,Cantonese Restaurant,Caucasian Restaurant,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Dairy Store,Dance Studio,Department Store,Design Studio,Dessert Shop,Dive Spot,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Exhibit,Fabric Shop,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Forest,Fountain,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Rink,Hookah Bar,Hot Dog Joint,Hotel,Ice Cream Shop,Indie Theater,Indoor Play Area,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Karaoke Bar,Kids Store,Lake,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Market,Martial Arts Dojo,Massage Studio,Medical Center,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Noodle House,Optical Shop,Other Great Outdoors,Other Repair Shop,Outdoor Gym,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pedestrian Plaza,Pelmeni House,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pizza Place,Playground,Plaza,Pool,Pool Hall,Print Shop,Pub,Public Art,Recording Studio,Recreation Center,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shawarma Place,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Spa,Sporting Goods Shop,Sports Club,Stables,Stadium,Strip Club,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Theater,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Tram Station,Tree,Vape Store,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Wine Shop,Women's Store,Yoga Studio
0,Филёвский Парк,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Филёвский Парк,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Филёвский Парк,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

And let's examine the new dataframe size.

In [38]:
moscow_onehot.shape

(854, 206)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [39]:
moscow_grouped = moscow_onehot.groupby('Neighborhood').mean().reset_index()


Let's confirm the new size.

In [40]:
moscow_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bakery,Bar,Bath House,Beach,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Blini House,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Burger Joint,Bus Line,Bus Station,Bus Stop,Butcher,Café,Camera Store,Campground,Cantonese Restaurant,Caucasian Restaurant,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Dairy Store,Dance Studio,Department Store,Design Studio,Dessert Shop,Dive Spot,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Exhibit,Fabric Shop,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Forest,Fountain,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Rink,Hookah Bar,Hot Dog Joint,Hotel,Ice Cream Shop,Indie Theater,Indoor Play Area,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Karaoke Bar,Kids Store,Lake,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Market,Martial Arts Dojo,Massage Studio,Medical Center,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Noodle House,Optical Shop,Other Great Outdoors,Other Repair Shop,Outdoor Gym,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pedestrian Plaza,Pelmeni House,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pizza Place,Playground,Plaza,Pool,Pool Hall,Print Shop,Pub,Public Art,Recording Studio,Recreation Center,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shawarma Place,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Spa,Sporting Goods Shop,Sports Club,Stables,Stadium,Strip Club,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Theater,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Tram Station,Tree,Vape Store,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Wine Shop,Women's Store,Yoga Studio
0,Алексеевский,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.107143,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035714,0.0,0.000000,0.0,0.0,0.0,0.035714,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.035714,0.0,0.0,0.035714,0.0,0.0,0.0,0.0,0.035714,0.0,0.000,0.0,0.0,0.000000,0.0,0.000000,0.035714,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035714,0.0,0.0,0.000,0.0,0.000000,0.071429,0.0,0.0,0.0,0.0,0.035714,0.0,0.0,0.0,0.0,0.0,0.000000,0.107143,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.035714,0.0,0.0,0.0,0.0,0.035714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.035714,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.071429,0.0,0.000000,0.0,0.0,0.000000,0.000,0.0,0.035714,0.0,0.0,0.035714,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.035714,0.000,0.0,0.0,0.0,0.035714,0.0,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035714
1,Алтуфьевский,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.125000,0.0,0.0,0.000000,0.0,0.000000,0

Let's put that into a *pandas* dataframe.

In [41]:
moscow_grouped.drop(columns=["Bus Line","Bus Stop"],inplace=True)

First, let's write a function to sort the venues in descending order.

In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [43]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = moscow_grouped['Neighborhood']


In [44]:
for ind in np.arange(moscow_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(moscow_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(15)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Алексеевский,Hotel,Auto Workshop,Pizza Place,Supermarket,Gym / Fitness Center,Yoga Studio,Farmers Market,Sports Club,Mobile Phone Shop,Snack Place
1,Алтуфьевский,Soccer Field,Pub,Lake,Dry Cleaner,Auto Workshop,Salon / Barbershop,Grocery Store,Exhibit,Food Court,Food & Drink Shop
2,Аэропорт,Pharmacy,Yoga Studio,Martial Arts Dojo,Smoke Shop,Jazz Club,Café,Clothing Store,Salon / Barbershop,Food Court,Food & Drink Shop
3,Бабушкинский,Cosmetics Shop,Pharmacy,Yoga Studio,Electronics Store,Boutique,Caucasian Restaurant,Clothing Store,Convenience Store,Plaza,Pet Store
4,Бескудниковский,Park,Plaza,Supermarket,Asian Restaurant,Fast Food Restaurant,Bar,Yoga Studio,Fountain,Forest,Food Court
5,Бибирево,Italian Restaurant,Department Store,Skating Rink,Pharmacy,Gym,Bar,Farmers Market,Forest,Food Court,Food & Drink Shop
6,Бутырский,Coffee Shop,Photography Studio,Kids Store,Martial Arts Dojo,Clothing Store,Accessories Store,Sushi Restaurant,Dance Studio,Music Venue,Boutique
7,Внуково,Soccer Field,Fast Food Restaurant,Scenic Lookout,Yoga Studio,Exhibit,Forest,Food Court,Food & Drink Shop,Flower Shop,Fish Market
8,Войковский,Clothing Store,Shoe Store,Lingerie Store,Sporting Goods Shop,Coffee Shop,Shopping Mall,Toy / Game Store,Seafood Restaurant,Garden Center,Bookstore
9,Восточное Дегунино,Supermarket,Bookstore,Pet Store,Miscellaneous Shop,Café,Gym,Baby Store,Dumpling Restaurant,Fish Market,Fountain


In [45]:
moscow_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bakery,Bar,Bath House,Beach,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Blini House,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Burger Joint,Bus Station,Butcher,Café,Camera Store,Campground,Cantonese Restaurant,Caucasian Restaurant,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Dairy Store,Dance Studio,Department Store,Design Studio,Dessert Shop,Dive Spot,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Exhibit,Fabric Shop,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Forest,Fountain,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Rink,Hookah Bar,Hot Dog Joint,Hotel,Ice Cream Shop,Indie Theater,Indoor Play Area,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Karaoke Bar,Kids Store,Lake,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Market,Martial Arts Dojo,Massage Studio,Medical Center,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Noodle House,Optical Shop,Other Great Outdoors,Other Repair Shop,Outdoor Gym,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pedestrian Plaza,Pelmeni House,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pizza Place,Playground,Plaza,Pool,Pool Hall,Print Shop,Pub,Public Art,Recording Studio,Recreation Center,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shawarma Place,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Spa,Sporting Goods Shop,Sports Club,Stables,Stadium,Strip Club,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Theater,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Tram Station,Tree,Vape Store,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Wine Shop,Women's Store,Yoga Studio
0,Алексеевский,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.107143,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.035714,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.035714,0.0,0.0,0.035714,0.0,0.0,0.0,0.0,0.035714,0.0,0.000,0.0,0.0,0.000000,0.0,0.000000,0.035714,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035714,0.0,0.0,0.000,0.0,0.000000,0.071429,0.0,0.0,0.0,0.0,0.035714,0.0,0.0,0.0,0.0,0.0,0.000000,0.107143,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.035714,0.0,0.0,0.0,0.0,0.035714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.035714,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.071429,0.0,0.000000,0.0,0.0,0.000000,0.000,0.0,0.035714,0.0,0.0,0.035714,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.035714,0.000,0.0,0.0,0.0,0.035714,0.0,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035714
1,Алтуфьевский,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.125000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000

<a id='item4'></a>

## Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 4 clusters.

In [46]:
# set number of clusters
kclusters = 4

moscow_grouped_clustering = moscow_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(moscow_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 3, 0, 2, 0, 0, 2, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [47]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

moscow_merged = moscow_df

# merge moscow_grouped with moscow_data to add latitude/longitude for each neighborhood
moscow_merged = moscow_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

moscow_merged 

,Neighborhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Филёвский Парк,ЗАО,55.748708,37.474149,0.0,Concert Hall,Athletics & Sports,Beach,BBQ Joint,Tea Room,Yoga Studio,Fast Food Restaurant,Frozen Yogurt Shop,Fountain,Forest
1,Внуково,ЗАО,55.612560,37.280081,2.0,Soccer Field,Fast Food Restaurant,Scenic Lookout,Yoga Studio,Exhibit,Forest,Food Court,Food & Drink Shop,Flower Shop,Fish Market
2,Щукино,СЗАО,55.798118,37.475900,0.0,Plaza,Coffee Shop,Dessert Shop,Sporting Goods Shop,Food & Drink Shop,Multiplex,Fast Food Restaurant,Boxing Gym,Fountain,Furniture / Home Store
3,Дмитровский,САО,55.886669,37.529184,0.0,Supermarket,Modern European Restaurant,Gym,Pizza Place,Restaurant,Pharmacy,Park,Eastern European Restaurant,Café,Soccer Field
4,Ховрино,САО,55.865166,37.486793,0.0,Supermarket,Gym / Fitness Center,Café,Pizza Place,Coffee Shop,Pet Store,Farmers Market,Sushi Restaurant,Grocery Store,Salon / Barbershop
5,Можайский,ЗАО,55.704913,37.383375,2.0,Fast Food Restaurant,Clothing Store,Cosmetics Shop,Big Box Store,Blini House,Mobile Phone Shop,Sushi Restaurant,Gym / Fitness Center,Boutique,Hot Dog Joint
6,Ново-Переделкино,ЗАО,55.650576,37.354337,2.0,Gym Pool,Tennis Court,Gym / Fitness Center,Spa,Yoga Studio,Forest,Food Court,Food & Drink Shop,Flower Shop,Fish Market
7,Строгино,СЗАО,55.797015,37.392717,0.0,Supermarket,Department Store,Tram Station,Bakery,Pedestrian Plaza,Toy / Game Store,Restaurant,Rental Car Location,Gym / Fitness Center,Pet Store
8,Молжаниновский,САО,55.943791,37.381299,0.0,Restaurant,Other Great Outdoors,Lake,Yoga Studio,Exhibit,Forest,Food Court,Food & Drink Shop,Flower Shop,Fish Market
9,Митино,СЗАО,55.855051,37.360428,2.0,Japanese Restaurant,Pedestrian Plaza,Gym Pool,Forest,Shopping Mall,Fast Food Restaurant,Middle Eastern Restaurant,Campground,Martial Arts Dojo,Sushi Restaurant


Finally, let's visualize the resulting clusters

In [48]:
moscow_merged['Cluster Labels'].value_counts()

0.0    28
2.0    15
3.0     4
1.0     3
Name: Cluster Labels, dtype: int64

Now let's create map of Moscow with clustered neighborhoods marked in different colors.

In [49]:
moscow_merged['Cluster Labels'].isna().value_counts()

False    50
True      1
Name: Cluster Labels, dtype: int64

In [50]:
moscow_merged.dropna(inplace=True)

In [51]:
moscow_merged=moscow_merged.astype({'Cluster Labels':'int32'})

In [62]:
# create map
map_clusters = folium.Map(location=[55.7, 37.4], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(moscow_merged['Latitude'], moscow_merged['Longitude'], moscow_merged['Neighborhood'], moscow_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## Examine Clusters

Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster.

#### Cluster 1

In [53]:
moscow_merged.loc[moscow_merged['Cluster Labels'] == 0, moscow_merged.columns[[0] + list(range(5, moscow_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Филёвский Парк,Concert Hall,Athletics & Sports,Beach,BBQ Joint,Tea Room,Yoga Studio,Fast Food Restaurant,Frozen Yogurt Shop,Fountain,Forest
2,Щукино,Plaza,Coffee Shop,Dessert Shop,Sporting Goods Shop,Food & Drink Shop,Multiplex,Fast Food Restaurant,Boxing Gym,Fountain,Furniture / Home Store
3,Дмитровский,Supermarket,Modern European Restaurant,Gym,Pizza Place,Restaurant,Pharmacy,Park,Eastern European Restaurant,Café,Soccer Field
4,Ховрино,Supermarket,Gym / Fitness Center,Café,Pizza Place,Coffee Shop,Pet Store,Farmers Market,Sushi Restaurant,Grocery Store,Salon / Barbershop
7,Строгино,Supermarket,Department Store,Tram Station,Bakery,Pedestrian Plaza,Toy / Game Store,Restaurant,Rental Car Location,Gym / Fitness Center,Pet Store
8,Молжаниновский,Restaurant,Other Great Outdoors,Lake,Yoga Studio,Exhibit,Forest,Food Court,Food & Drink Shop,Flower Shop,Fish Market
14,Южное Тушино,Park,Cosmetics Shop,Gastropub,Gym,Supermarket,Karaoke Bar,Arts & Crafts Store,Convenience Store,Wine Shop,Bath House
17,Хорошёво-Мнёвники,Coffee Shop,Caucasian Restaurant,Italian Restaurant,Sushi Restaurant,Kids Store,Massage Studio,Gourmet Shop,Modern European Restaurant,Fast Food Restaurant,Park
19,Тропарёво-Никулино,Coffee Shop,Cosmetics Shop,Bakery,Gym / Fitness Center,Multiplex,Noodle House,Bookstore,Middle Eastern Restaurant,Blini House,Mexican Restaurant
20,Левобережный,Sporting Goods Shop,Convenience Store,Café,Sushi Restaurant,Film Studio,Liquor Store,Fish Market,Supermarket,Climbing Gym,Big Box Store


#### Cluster 2

In [54]:
moscow_merged.loc[moscow_merged['Cluster Labels'] == 1, moscow_merged.columns[[0] + list(range(5, moscow_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Куркино,Park,Restaurant,Scenic Lookout,Yoga Studio,Exhibit,Forest,Food Court,Food & Drink Shop,Flower Shop,Fish Market
21,Фили-Давыдково,Park,Italian Restaurant,Gym / Fitness Center,Hotel,Restaurant,Dog Run,Athletics & Sports,Food Court,Food & Drink Shop,Flower Shop
29,Северный,Fountain,Park,Eastern European Restaurant,Yoga Studio,Fabric Shop,Forest,Food Court,Food & Drink Shop,Flower Shop,Fish Market


#### Cluster 3

In [55]:
moscow_merged.loc[moscow_merged['Cluster Labels'] == 2, moscow_merged.columns[[0] + list(range(5, moscow_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Внуково,Soccer Field,Fast Food Restaurant,Scenic Lookout,Yoga Studio,Exhibit,Forest,Food Court,Food & Drink Shop,Flower Shop,Fish Market
5,Можайский,Fast Food Restaurant,Clothing Store,Cosmetics Shop,Big Box Store,Blini House,Mobile Phone Shop,Sushi Restaurant,Gym / Fitness Center,Boutique,Hot Dog Joint
6,Ново-Переделкино,Gym Pool,Tennis Court,Gym / Fitness Center,Spa,Yoga Studio,Forest,Food Court,Food & Drink Shop,Flower Shop,Fish Market
9,Митино,Japanese Restaurant,Pedestrian Plaza,Gym Pool,Forest,Shopping Mall,Fast Food Restaurant,Middle Eastern Restaurant,Campground,Martial Arts Dojo,Sushi Restaurant
11,Крылатское,Pizza Place,Athletics & Sports,Auto Workshop,Track,Park,Yoga Studio,Fabric Shop,Fountain,Forest,Food Court
12,Солнцево,Recreation Center,Convenience Store,Food & Drink Shop,Pizza Place,Supermarket,Auto Workshop,Yoga Studio,Fabric Shop,Forest,Food Court
13,Головинский,Food & Drink Shop,Supermarket,Park,Stadium,Playground,Pizza Place,Middle Eastern Restaurant,Gourmet Shop,Exhibit,Flower Shop
16,Покровское-Стрешнево,Fast Food Restaurant,Soccer Stadium,Auto Workshop,Farmers Market,Gun Range,Market,Gym / Fitness Center,Blini House,Clothing Store,Park
18,Очаково-Матвеевское,Hotel,Fast Food Restaurant,Lighthouse,Auto Workshop,Park,Pet Store,Yoga Studio,Fabric Shop,Forest,Food Court
30,Бескудниковский,Park,Plaza,Supermarket,Asian Restaurant,Fast Food Restaurant,Bar,Yoga Studio,Fountain,Forest,Food Court


#### Cluster 4

In [56]:
moscow_merged.loc[moscow_merged['Cluster Labels'] == 3, moscow_merged.columns[[0] + list(range(5, moscow_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Северное Тушино,Convenience Store,Pharmacy,Gym / Fitness Center,Wine Shop,Beer Store,Café,Grocery Store,Park,Yoga Studio,Food Court
27,Коптево,Café,Convenience Store,Pharmacy,Farmers Market,Park,Middle Eastern Restaurant,Gym / Fitness Center,Big Box Store,Hardware Store,Exhibit
28,Аэропорт,Pharmacy,Yoga Studio,Martial Arts Dojo,Smoke Shop,Jazz Club,Café,Clothing Store,Salon / Barbershop,Food Court,Food & Drink Shop
36,Марфино,Botanical Garden,Gym / Fitness Center,Pool,Convenience Store,Arts & Entertainment,Sushi Restaurant,Fountain,Forest,Food Court,Food & Drink Shop


## Conclusion

The neighborhood I currently reside in is located in the Cluster 0. 
This cluster is characterized by a lot of shops, cafes and restaurants.
So, as possible neighborhoods for relocation I could select other neighborhoods from that cluster, for example Сокол or Хорошёво-Мневники.